In [4]:
import torch
import os

print(f"PyTorch Version: {torch.__version__}")
print(f"Current Directory: {os.getcwd()}")
print(f"Files in Directory: {os.listdir('.')}")


PyTorch Version: 2.6.0+cu124
Current Directory: C:\Users\arche\PycharmProjects\yt-viral-predictor
Files in Directory: ['.git', '.gitignore', '.idea', '.ipynb_checkpoints', '.venv', 'data', 'docker-compose.gpu.yml', 'docker-compose.yml', 'Dockerfile', 'download.py', 'LICENSE', 'main.py', 'nb.ipynb', 'README.md', 'requirements.gpu.txt', 'requirements.txt']


In [5]:
print(torch.cuda.is_available())

True


In [2]:
with open("api.txt", "r") as f:
    api_key = f.read().strip()


import os
import pandas as pd

import googleapiclient.discovery
import googleapiclient.errors

from IPython.display import JSON

scopes = ["https://www.googleapis.com/auth/youtube.readonly"]

def main(id_list):
    # Disable OAuthlib's HTTPS verification when running locally.
    # *DO NOT* leave this option enabled in production.
    os.environ["OAUTHLIB_INSECURE_TRANSPORT"] = "1"

    api_service_name = "youtube"
    api_version = "v3"

    youtube = googleapiclient.discovery.build(
        api_service_name, api_version, developerKey=api_key)

    request = youtube.channels().list(
        part="snippet,contentDetails,statistics",
        id=','.join(id_list)
    )
    response = request.execute()

    return JSON(response)

channel_ids = [
    'UCgKFOz_KrMbmypWrawtzDQg',
]

display(main(channel_ids))

<IPython.core.display.JSON object>

In [3]:
import requests
from googleapiclient.discovery import build

API_KEY = api_key
VIDEO_ID = "3fpqMJCoUQ0" # Example ID

def get_thumbnail_api(video_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)

    # Request video details
    request = youtube.videos().list(
        part="snippet",
        id=video_id
    )
    response = request.execute()

    # Extract the highest resolution URL available
    thumbnails = response['items'][0]['snippet']['thumbnails']

    # Try to get maxres, then high, then default
    if 'maxres' in thumbnails:
        return thumbnails['maxres']['url']
    return thumbnails.get('high', {}).get('url')

thumbnail_url = get_thumbnail_api(VIDEO_ID, API_KEY)
print(f"Found URL: {thumbnail_url}")


def download_image(url, filename="thumbnail.jpg"):
    response = requests.get(url)
    if response.status_code == 200:
        with open(filename, 'wb') as f:
            f.write(response.content)
        print(f"Success! Image saved as {filename}")
    else:
        print("Failed to download image.")


Success! Image saved as my_video_thumbnail.jpg


In [5]:
download_image(thumbnail_url, "data/raw/thumbnails/test.jpg")

Success! Image saved as data/raw/thumbnails/test.jpg


In [7]:
from googleapiclient.discovery import build

PLAYLIST_ID = "PLcEN1HW6GYIczNHmEUSKkJ646Ev3-i0A2"

def get_video_ids_from_playlist(playlist_id, api_key):
    youtube = build("youtube", "v3", developerKey=api_key)
    video_ids = []
    next_page_token = None

    while True:
        # Request a page of playlist items
        request = youtube.playlistItems().list(
            part="contentDetails",
            playlistId=playlist_id,
            maxResults=50,
            pageToken=next_page_token
        )
        response = request.execute()

        # Extract Video IDs from the current page
        for item in response.get('items', []):
            video_ids.append(item['contentDetails']['videoId'])

        # Check if there's another page
        next_page_token = response.get('nextPageToken')

        if not next_page_token:
            break

    return video_ids

# Usage
all_ids = get_video_ids_from_playlist(PLAYLIST_ID, API_KEY)
print(f"Retrieved {len(all_ids)} video IDs.")
print(all_ids[:5])  # Show first five

Retrieved 12 video IDs.
['xx4vDZdRWDU', 'CHzha4Fxlg0', 'JGWabJWC24U', 'gaKUpWq5Qas', 'HWRZRfmrxRc']


In [8]:
for id in all_ids:
    download_image(get_thumbnail_api(id, API_KEY), f"data/raw/thumbnails/{id}.jpg")

Success! Image saved as data/raw/thumbnails/4lchhpLHmaA.jpg
